In [1]:
import os
import numpy as np
from PIL import Image

IMAGE_SIZE = 128
IMAGE_CHANNELS = 3
IMAGE_DIR = '/home/ubuntu/Desktop/Image2'

images_path = IMAGE_DIR

In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib

In [11]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17062434975717288583
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17171842580870427056
physical_device_desc: "device: XLA_CPU device"
]


In [3]:

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [23]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [3]:
training_data = []

# Iterating over the images inside the directory and resizing them using
# Pillow's resize method.
print('resizing...')

for filename in os.listdir(images_path):
    path = os.path.join(images_path, filename)
    image = Image.open(path).resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
    #print ('hello')

    training_data.append(np.asarray(image))

training_data = np.reshape(
    training_data, (-1, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
training_data = training_data / 127.5 - 1

print('saving file...')
np.save('cubism_data.npy', training_data)

resizing...
saving file...


In [4]:
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import numpy as np
from PIL import Image
import os

In [5]:
# Preview image Frame
PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 4
SAVE_FREQ = 100 # Size vector to generate images from
NOISE_SIZE = 100 # Configuration
EPOCHS = 3000 # number of iterations
BATCH_SIZE = 32
GENERATE_RES = 3
IMAGE_SIZE = 128 # rows/colsIMAGE_CHANNELS = 3

In [6]:
training_data = np.load('cubism_data.npy')

In [7]:
def build_discriminator(image_shape):    
    model = Sequential()    
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))    
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))    
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))    
    model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))    
    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))    
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))    
    input_image = Input(shape=image_shape)
    validity = model(input_image)
    return Model(input_image, validity)

In [8]:
def build_generator(noise_size, channels):
    model = Sequential()
    model.add(Dense(4 * 4 * 256, activation='relu', input_dim=noise_size))
    model.add(Reshape((4, 4, 256)))    
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))    
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))    
    for i in range(GENERATE_RES):
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation('relu'))    
    model.summary()
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation('tanh'))    
    input = Input(shape=(noise_size,))
    generated_image = model(input)
    
    return Model(input, generated_image)

In [22]:
def save_images(cnt, noise):
    image_array = np.full((
        PREVIEW_MARGIN + (PREVIEW_ROWS * (IMAGE_SIZE + PREVIEW_MARGIN)),
        PREVIEW_MARGIN + (PREVIEW_COLS * (IMAGE_SIZE + PREVIEW_MARGIN)), 3),
        255, dtype=np.uint8)
    
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5
    image_count = 0
    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
            r = row * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            c = col * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            image_array[r:r + IMAGE_SIZE, c:c +
                        IMAGE_SIZE] = generated_images[image_count] * 255
            image_count += 1
            output_path = 'output'
    filename = os.path.join(output_path, f"trained-{cnt}.png")
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        filename = os.path.join(output_path, f"trained-{cnt}.png")
    im = Image.fromarray(image_array)
    im.save(filename)

In [17]:
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)
optimizer = Adam(1.5e-4, 0.5)
discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy',
optimizer=optimizer, metrics=['accuracy'])

In [18]:
generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)
random_input = Input(shape=(NOISE_SIZE,))
generated_image = generator(random_input)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 4096)              413696    
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 256)         0         
_________________________________________________________________
up_sampling2d_5 (UpSampling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 8, 8, 256)         590080    
_________________________________________________________________
batch_normalization_13 (Batc (None, 8, 8, 256)         1024      
_________________________________________________________________
activation_6 (Activation)    (None, 8, 8, 256)         0         
_________________________________________________________________
up_sampling2d_6 (UpSampling2 (None, 16, 16, 256)      

In [19]:
discriminator.trainable = False
validity = discriminator(generated_image)
combined = Model(random_input, validity)
combined.compile(loss='binary_crossentropy',
optimizer=optimizer, metrics=['accuracy'])

In [20]:
y_real = np.ones((BATCH_SIZE, 1))
y_fake = np.zeros((BATCH_SIZE, 1))
fixed_noise = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, NOISE_SIZE))

In [23]:
cnt = 1
for epoch in range(EPOCHS):
    idx = np.random.randint(0, training_data.shape[0], BATCH_SIZE)
    x_real = training_data[idx]
 
    noise= np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE))
    x_fake = generator.predict(noise)
 
    discriminator_metric_real = discriminator.train_on_batch(x_real, y_real)  
    discriminator_metric_generated = discriminator.train_on_batch(
 x_fake, y_fake)
 
    discriminator_metric = 0.5 * np.add(discriminator_metric_real,   discriminator_metric_generated)
    generator_metric = combined.train_on_batch(noise, y_real)
    if epoch % SAVE_FREQ == 0:
        save_images(cnt, fixed_noise)
        cnt += 1
 
        print(f"{epoch} epoch, Discriminator accuracy: {100*  discriminator_metric[1]}, Generator accuracy: {100 * generator_metric[1]}")

0 epoch, Discriminator accuracy: 59.375, Generator accuracy: 0.0
100 epoch, Discriminator accuracy: 51.5625, Generator accuracy: 81.25
200 epoch, Discriminator accuracy: 100.0, Generator accuracy: 100.0
300 epoch, Discriminator accuracy: 100.0, Generator accuracy: 100.0
400 epoch, Discriminator accuracy: 100.0, Generator accuracy: 100.0
500 epoch, Discriminator accuracy: 100.0, Generator accuracy: 100.0
600 epoch, Discriminator accuracy: 100.0, Generator accuracy: 100.0
700 epoch, Discriminator accuracy: 100.0, Generator accuracy: 100.0
800 epoch, Discriminator accuracy: 100.0, Generator accuracy: 100.0
900 epoch, Discriminator accuracy: 100.0, Generator accuracy: 100.0
1000 epoch, Discriminator accuracy: 100.0, Generator accuracy: 100.0
1100 epoch, Discriminator accuracy: 100.0, Generator accuracy: 100.0
1200 epoch, Discriminator accuracy: 100.0, Generator accuracy: 100.0
1300 epoch, Discriminator accuracy: 100.0, Generator accuracy: 100.0
1400 epoch, Discriminator accuracy: 100.0, Ge

In [24]:
discriminator_metric_real = discriminator.train_on_batch(x_real, y_real)
discriminator_metric_generated = discriminator.train_on_batch(x_fake, y_fake)

In [25]:
discriminator_metric = 0.5 * np.add(discriminator_metric_real, discriminator_metric_generated)

In [54]:
print("Shape: ", generated_image[0].shape)
tf.summary.image("Training data", generated_image, step=0)


Shape:  (128, 128, 3)


<tf.Tensor: shape=(), dtype=bool, numpy=False>